## Final Project
### Data Science LHL Project: Restaurant Reccomenation system

### Part 1: Data Collection

By: Chloe Phuong

In [8]:
import numpy as np
import pandas as pd
import requests

#### Request API for Bike Station

In [ ]:
# CityBikes

# API endpoint for Toronto city
url = 'https://api.citybik.es/v2/networks/toronto'

# Send GET request to retrieve bike-sharing data for Toronto
response = requests.get(url)

# Extract the data from the response
data = response.json()

# Extract the list of stations
stations = data['network']['stations']

# Iterate over each station and access the details
for i, station in enumerate(stations):
    if i > 100:
        break  # Exit the loop if the limit is reached
    
    station_id = station.get('id')
    name = station.get('name')
    latitude = station.get('latitude')
    longitude = station.get('longitude')

# Create a list to store the station details
station_details = []

# Iterate over each station and access the details
for i, station in enumerate(stations):
    if i > 100:
        break  # Exit the loop if the limit is reached
    
    station_id = station['id']
    name = station['name']
    latitude = station['latitude']
    longitude = station['longitude']

    # Append the station details as a dictionary to the list
    station_details.append({
        'Station ID': station_id,
        'Name': name,
        'Latitude': latitude,
        'Longitude': longitude,
    })

# Create the bike stations DataFrame
bike_df = pd.DataFrame(station_details)
bike_df

In [10]:
# Save bike_df to json file
bike_df.to_json('bike_df')

In [18]:
bike_df = pd.read_json('bike_df')

bike_df

,Station ID,Name,Latitude,Longitude
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947
2,a09c67c0b419654d907c9134b108e328,Queen St. E / Rhodes Ave.,43.666224,-79.317693
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288
...,...,...,...,...
96,908b17c0d06011373777bace0a3cd74c,Beltline Trail / Chaplin Cres.,43.704461,-79.423122
97,d2ce6ae839b7df2c912ab140253a5574,Lake Shore Blvd E / Leslie St,43.659195,-79.329196
98,1d35b8fd44d73aabbb39f6e142ee3e7e,King St E / Berkeley St,43.651885,-79.364900
99,72de8a8e0a0966768f8115d755532e4c,Rosedale Park (20 Scholfield Ave),43.682645,-79.377557


#### Request Yelp API

Request API for Restaurants

In [21]:
API_key = '-j9DtFiL0178Mkmpjj6H-J9tCkhh-jkJwU1jak7K3Dyf690N-ucWrr0pEOOWlh9K-4RkGhoM762HRCS8EDacM3f-8q4gqx4iIPV4oTrkNjRqCSg711SBUWcyB0uqZHYx'
url = 'https://api.yelp.com/v3/businesses/search'
radius = 1000  # Radius in meters
limit = 10

In [22]:
# Create an empty list to store the extracted details
rest_details = []

# Iterate over each row in the dataframe
for index, row in bike_df.iterrows():
    station_id = row['Station ID']
    latitude = row['Latitude']
    longitude = row['Longitude']

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'term': 'restaurants',
        'limit': limit
    }
    headers = {
        'Authorization': f'Bearer {API_key}'
    }

    # Send API request to Yelp
    response = requests.get(url, params=params, headers=headers)
    rest_data = response.json()

    # Check if 'businesses' key exists in the response
    if 'businesses' in rest_data:
        businesses = rest_data['businesses']

        # Iterate over each business and extract the details
        for business in businesses:
            business_id = business['id']
            name = business['name']
            address = ', '.join(business['location']['display_address'])
            city = business['location']['city']
            postal_code = business['location']['zip_code']
            latitude = business['coordinates']['latitude']
            longitude = business['coordinates']['longitude']
            stars = business['rating']
            review_count = business['review_count']
            is_open = not business['is_closed']
            categories = business['categories']
            hours = business.get('hours')

            # Append the restaurant details to the list
            rest_details.append({
                'business_id': business_id,
                'name': name,
                'address': address,
                'city': city,
                'postal_code': postal_code,
                'latitude': latitude,
                'longitude': longitude,
                'stars': stars,
                'review_count': review_count,
                'is_open': is_open,
                'categories': categories,
                'hours': hours
            })

# Create a DataFrame from the list of restaurant details
rest_df = pd.DataFrame(rest_details)

# Print the DataFrame
rest_df

,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
0,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.666270,-79.318020,4.5,15,True,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",None
1,Huo1lJmVkdqvFuLtA74Nhw,Casa Di Giorgios,"1646 Queen St E, Toronto, ON M4L 1G3, Canada",Toronto,M4L 1G3,43.666850,-79.315180,4.0,75,True,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",None
2,yP71_LfUPnGIRqA2g09RoQ,Lake Inez,"1471 Gerrard Street E, Toronto, ON M4L 2A1, Ca...",Toronto,M4L 2A1,43.672340,-79.320640,4.5,94,True,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",None
3,TFpo3-XpYTdcz_LGjcHrdQ,Mattachioni,"1501 Gerrard St E, Toronto, ON M4L 2A4, Canada",Toronto,M4L 2A4,43.672640,-79.319730,4.5,5,True,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",None
4,Zru9ASPdsLSjfIlpozgeMA,Puerto Bravo,"1425 Gerrard Street E, Toronto, ON M4L 1Z7, Ca...",Toronto,M4L 1Z7,43.672030,-79.322310,4.5,4,True,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",None
...,...,...,...,...,...,...,...,...,...,...,...,...
999,qPVqfJAzC8APF8aWGBtFRA,Antler Kitchen & Bar,"1454 Dundas Street West, Toronto, ON M6J 1Y6, ...",Toronto,M6J 1Y6,43.649850,-79.430590,4.5,289,True,"[{'alias': 'newcanadian', 'title': 'Canadian (...",None
1000,xIKmVF_kRG_NDsmg0xmzTw,Nuna Kitchen & Bar,"1265 Queen Street W, Toronto, ON M6K 1L4, Canada",Toronto,M6K 1L4,43.641540,-79.431120,4.5,19,True,"[{'alias': 'international', 'title': 'Internat...",None
1001,YHubCF0Ua0uC6HC2tm-arQ,The Federal,"1438 Dundas Street W, Toronto, ON M6J 3L5, Canada",Toronto,M6J 3L5,43.649770,-79.430110,4.0,173,True,"[{'alias': 'breakfast_brunch', 'title': 'Break...",None
1002,6hGEIpALyUhte4grCY8Hog,The Opium Bar,"1321 Dundas Street West, Toronto, ON M6J 1X8, ...",Toronto,M6J 1X8,43.649280,-79.426280,4.5,20,True,"[{'alias': 'whiskeybars', 'title': 'Whiskey Ba...",None


In [23]:
rest_df.shape

(1004, 12)

In [38]:
rest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004 entries, 0 to 1003
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   1004 non-null   object 
 1   name          1004 non-null   object 
 2   address       1004 non-null   object 
 3   city          1004 non-null   object 
 4   postal_code   1004 non-null   object 
 5   latitude      1004 non-null   float64
 6   longitude     1004 non-null   float64
 7   stars         1004 non-null   float64
 8   review_count  1004 non-null   int64  
 9   is_open       1004 non-null   bool   
 10  categories    1004 non-null   object 
 11  hours         0 non-null      object 
dtypes: bool(1), float64(3), int64(1), object(7)
memory usage: 87.4+ KB


In [25]:
# Save rest_df to json file
rest_df.to_json('rest_df')

In [26]:
rest_data_df = pd.read_json('rest_df')

rest_df

,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
0,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.666270,-79.318020,4.5,15,True,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",None
1,Huo1lJmVkdqvFuLtA74Nhw,Casa Di Giorgios,"1646 Queen St E, Toronto, ON M4L 1G3, Canada",Toronto,M4L 1G3,43.666850,-79.315180,4.0,75,True,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",None
2,yP71_LfUPnGIRqA2g09RoQ,Lake Inez,"1471 Gerrard Street E, Toronto, ON M4L 2A1, Ca...",Toronto,M4L 2A1,43.672340,-79.320640,4.5,94,True,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",None
3,TFpo3-XpYTdcz_LGjcHrdQ,Mattachioni,"1501 Gerrard St E, Toronto, ON M4L 2A4, Canada",Toronto,M4L 2A4,43.672640,-79.319730,4.5,5,True,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",None
4,Zru9ASPdsLSjfIlpozgeMA,Puerto Bravo,"1425 Gerrard Street E, Toronto, ON M4L 1Z7, Ca...",Toronto,M4L 1Z7,43.672030,-79.322310,4.5,4,True,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",None
...,...,...,...,...,...,...,...,...,...,...,...,...
999,qPVqfJAzC8APF8aWGBtFRA,Antler Kitchen & Bar,"1454 Dundas Street West, Toronto, ON M6J 1Y6, ...",Toronto,M6J 1Y6,43.649850,-79.430590,4.5,289,True,"[{'alias': 'newcanadian', 'title': 'Canadian (...",None
1000,xIKmVF_kRG_NDsmg0xmzTw,Nuna Kitchen & Bar,"1265 Queen Street W, Toronto, ON M6K 1L4, Canada",Toronto,M6K 1L4,43.641540,-79.431120,4.5,19,True,"[{'alias': 'international', 'title': 'Internat...",None
1001,YHubCF0Ua0uC6HC2tm-arQ,The Federal,"1438 Dundas Street W, Toronto, ON M6J 3L5, Canada",Toronto,M6J 3L5,43.649770,-79.430110,4.0,173,True,"[{'alias': 'breakfast_brunch', 'title': 'Break...",None
1002,6hGEIpALyUhte4grCY8Hog,The Opium Bar,"1321 Dundas Street West, Toronto, ON M6J 1X8, ...",Toronto,M6J 1X8,43.649280,-79.426280,4.5,20,True,"[{'alias': 'whiskeybars', 'title': 'Whiskey Ba...",None


Request API for Reviews

In [36]:
API_key = 'qekbhZMmgf57ynmxgZU-d7KA7OSmD-7BjDKsYH3hpcq-QBW2DgnDaQ-tA788OeYrgu_TLbmQKzx6NbyBStHJ-CvOyo3Go1Ok9aM458sYJ_AtBFdb6f1K9h9f1xB8ZHYx'
reviews_url = 'https://api.yelp.com/v3/businesses/{business_id_or_alias}/reviews' # Define the API endpoint

In [37]:
# Create an empty list to store the reviews
all_reviews = []

In [39]:
# Iterate over each row in the DataFrame
for index, row in rest_df.iterrows():
    business_id = row['business_id']  # Retrieve the business_id from the DataFrame
    
    # Set the URL with the business ID
    url = reviews_url.replace('{business_id_or_alias}', business_id)
    
    # Set the headers with the API key
    headers = {
        'Authorization': f'Bearer {API_key}'
    }
    
    # Send the API request to Yelp
    response = requests.get(url, headers=headers)
    data = response.json()
    
    # Check if 'reviews' key exists in the response
    if 'reviews' in data:
        reviews = data['reviews']
        
        # Iterate over each review and extract the desired details
        for review in reviews:
            review_id = review['id']
            user_id = review['user']['id']
            stars_left = review['rating']
            text = review['text']
            date = review['time_created']
            
            # Append the review details as a dictionary to the list
            all_reviews.append({
                'business_id': business_id,
                'review_id': review_id,
                'user_id': user_id,
                'stars_left': stars_left,
                'text': text,
                'date': date
            })

# Create a DataFrame from the list of reviews
reviews_df = pd.DataFrame(all_reviews)

In [40]:
reviews_df

,business_id,review_id,user_id,stars_left,text,date
0,Ohengzi4viLEXqhK50aLxg,sR9e_G0JdycDTGV4S-Ma2w,qXztukjlJFpu3T-EmSWIIQ,5,"Incredible, the chicken is so tender, seasonin...",2023-02-12 11:07:53
1,Ohengzi4viLEXqhK50aLxg,ArpqFM_KEDrP0fJScO09Zw,KhpA23RweKsO9TDaGpE2xg,4,We were able to check this place out yesterday...,2022-04-25 12:51:22
2,Ohengzi4viLEXqhK50aLxg,w56U5WwwCzd2Sz9aXsuXjQ,XbiKsujS_qxU3xsr0xUqmQ,4,I think this will be our go-to before any even...,2022-06-03 10:01:45
3,Huo1lJmVkdqvFuLtA74Nhw,2suWW8X7h1ebvnXEFIupkw,JxTE65Qm5_r7zhpp3u609w,5,There aren't many places you can get good pizz...,2023-05-20 17:01:49
4,Huo1lJmVkdqvFuLtA74Nhw,yAaxlz-0mgcJIgu7UuLZiA,Aa5bZsoNRHgYkkw193Wu6A,3,Been going to Giorgio's for years. Not sure wh...,2023-02-03 07:22:41
...,...,...,...,...,...,...
1472,fN_I3jP7RD2llubTvhXtKQ,5q1tz1R-BIEkfB-l2RnwmQ,6x435xUBuB0MrgIEcPv7gw,4,I was so happy to come here for brunch! I enjo...,2023-04-29 09:38:59
1473,fN_I3jP7RD2llubTvhXtKQ,3I82uRXafiZ_xxB3Vtp-7w,f-PtTWeEf3RIDmR_jdRbWQ,5,This was a great find in Toronto! I had an alm...,2023-03-08 15:17:52
1474,wSojc-y-d7MWiGWdy8deCg,DUc0BbKOM-762xNQcpn93w,vztkm1IriIJ-GWg4v5o8-Q,4,A great spot for BBQ located on the bustling R...,2023-05-01 19:31:27
1475,wSojc-y-d7MWiGWdy8deCg,pTlZFUrgNNsKYYTZAiCJmw,tIMuKya7GAvlNsBj_Cpifg,3,Scattered thoughts about this place. Atmospher...,2023-04-02 18:17:28


In [41]:
reviews_df.shape

(1477, 6)

In [42]:
# Save reviews_df to json file
reviews_df.to_json('reviews_df')

In [53]:
reviews_data_df = pd.read_json('reviews_df')

reviews_df

,business_id,review_id,user_id,stars_left,text,date
0,Ohengzi4viLEXqhK50aLxg,sR9e_G0JdycDTGV4S-Ma2w,qXztukjlJFpu3T-EmSWIIQ,5,"Incredible, the chicken is so tender, seasonin...",2023-02-12 11:07:53
1,Ohengzi4viLEXqhK50aLxg,ArpqFM_KEDrP0fJScO09Zw,KhpA23RweKsO9TDaGpE2xg,4,We were able to check this place out yesterday...,2022-04-25 12:51:22
2,Ohengzi4viLEXqhK50aLxg,w56U5WwwCzd2Sz9aXsuXjQ,XbiKsujS_qxU3xsr0xUqmQ,4,I think this will be our go-to before any even...,2022-06-03 10:01:45
3,Huo1lJmVkdqvFuLtA74Nhw,2suWW8X7h1ebvnXEFIupkw,JxTE65Qm5_r7zhpp3u609w,5,There aren't many places you can get good pizz...,2023-05-20 17:01:49
4,Huo1lJmVkdqvFuLtA74Nhw,yAaxlz-0mgcJIgu7UuLZiA,Aa5bZsoNRHgYkkw193Wu6A,3,Been going to Giorgio's for years. Not sure wh...,2023-02-03 07:22:41
...,...,...,...,...,...,...
1472,fN_I3jP7RD2llubTvhXtKQ,5q1tz1R-BIEkfB-l2RnwmQ,6x435xUBuB0MrgIEcPv7gw,4,I was so happy to come here for brunch! I enjo...,2023-04-29 09:38:59
1473,fN_I3jP7RD2llubTvhXtKQ,3I82uRXafiZ_xxB3Vtp-7w,f-PtTWeEf3RIDmR_jdRbWQ,5,This was a great find in Toronto! I had an alm...,2023-03-08 15:17:52
1474,wSojc-y-d7MWiGWdy8deCg,DUc0BbKOM-762xNQcpn93w,vztkm1IriIJ-GWg4v5o8-Q,4,A great spot for BBQ located on the bustling R...,2023-05-01 19:31:27
1475,wSojc-y-d7MWiGWdy8deCg,pTlZFUrgNNsKYYTZAiCJmw,tIMuKya7GAvlNsBj_Cpifg,3,Scattered thoughts about this place. Atmospher...,2023-04-02 18:17:28


#### Combined Dataframe

In [69]:
# Merge the rest_df and reviews_df DataFrames
restrev_df = pd.merge(rest_df, reviews_df, on='business_id', how='left')

# Print the combined DataFrame
restrev_df.head()

,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours,review_id,user_id,stars_left,text,date
0,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",None,sR9e_G0JdycDTGV4S-Ma2w,qXztukjlJFpu3T-EmSWIIQ,5.0,"Incredible, the chicken is so tender, seasonin...",2023-02-12 11:07:53
1,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",None,ArpqFM_KEDrP0fJScO09Zw,KhpA23RweKsO9TDaGpE2xg,4.0,We were able to check this place out yesterday...,2022-04-25 12:51:22
2,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",None,w56U5WwwCzd2Sz9aXsuXjQ,XbiKsujS_qxU3xsr0xUqmQ,4.0,I think this will be our go-to before any even...,2022-06-03 10:01:45
3,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",None,sR9e_G0JdycDTGV4S-Ma2w,qXztukjlJFpu3T-EmSWIIQ,5.0,"Incredible, the chicken is so tender, seasonin...",2023-02-12 11:07:53
4,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",None,ArpqFM_KEDrP0fJScO09Zw,KhpA23RweKsO9TDaGpE2xg,4.0,We were able to check this place out yesterday...,2022-04-25 12:51:22


In [71]:
# Save restrev_df to json file
restrev_df.to_json('restrev_df')